In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
spark.conf.set(
    "fs.azure.account.key.parisolympicdataaristide.dfs.core.windows.net","<<key_value")

In [0]:
from pyspark.sql import SparkSession

In [0]:

display(dbutils.fs.ls("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data"))


path,name,size,modificationTime
abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/athletes.csv,athletes.csv,7156043,1736457536000
abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/coaches.csv,coaches.csv,93472,1736457566000
abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/medals.csv,medals.csv,179036,1736457603000
abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/medals_totals.csv,medals_totals.csv,3065,1736457583000
abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/teams.csv,teams.csv,442798,1736457551000


In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/coaches.csv")
total_medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/medals_totals.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/raw-data/teams.csv")
     

In [0]:
athletes.show(20)

+-------+-------+--------------------+---------------+--------------------+------+--------+------------+--------+------------+-----------+----------------+----------------+------+------+--------------------+--------------------+--------------------+-------------------+-------------+-------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   code|current|                name|     name_short|             name_tv|gender|function|country_code| country|country_long|nationality|nationality_long|nationality_code|height|weight|         disciplines|              events|          birth_date|        birth_place|birth_country|    residence_place|residence_country|            nickname|             hobbies|          occupation|

In [0]:
athletes.count()


11113

In [0]:
athletes.printSchema()

root
 |-- code: integer (nullable = true)
 |-- current: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- name_short: string (nullable = true)
 |-- name_tv: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- function: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- nationality_long: string (nullable = true)
 |-- nationality_code: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- disciplines: string (nullable = true)
 |-- events: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- birth_place: string (nullable = true)
 |-- birth_country: string (nullable = true)
 |-- residence_place: string (nullable = true)
 |-- residence_country: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- hobbies: string (nullable =

In [0]:
athletes = athletes.withColumn("birth_date",col("name").cast(DateType()))

In [0]:
coaches.printSchema(20)

root
 |-- code: integer (nullable = true)
 |-- current: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- function: string (nullable = true)
 |-- category: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- disciplines: string (nullable = true)
 |-- events: string (nullable = true)
 |-- birth_date: date (nullable = true)



In [0]:
display(coaches.limit(10))

code,current,name,gender,function,category,country_code,country,country_long,disciplines,events,birth_date
1533246,true,PEDRERO Ofelia,Female,Coach,C,MEX,Mexico,Mexico,Artistic Swimming,Team,1988-03-28
1535775,true,RADHI SHENAISHIL,Male,Head Coach,C,IRQ,Iraq,Iraq,Football,Men,1965-07-01
1536055,true,AFLAKIKHAMSEH Majid,Male,Coach,C,IRI,IR Iran,Islamic Republic of Iran,Taekwondo,null,1973-08-26
1536059,true,YOUSEFY Mehrdad,Male,Coach,C,IRI,IR Iran,Islamic Republic of Iran,Taekwondo,null,1972-06-12
1536060,true,MADDAH Minoo,Female,Coach,C,IRI,IR Iran,Islamic Republic of Iran,Taekwondo,null,1976-05-17
1536328,true,LOFTUS Adriana,Female,Coach,C,MEX,Mexico,Mexico,Artistic Swimming,Team,1958-06-28
1538313,true,FERRARA Fernando,Male,Head Coach,C,ARG,Argentina,Argentina,Hockey,null,1968-07-24
1538315,true,GULLA Alejandra,Female,Assistant Coach,C,ARG,Argentina,Argentina,Hockey,Women,1977-07-04
1538317,true,CAPURRO Santiago,Male,Assistant Coach,C,ARG,Argentina,Argentina,Hockey,null,1975-04-08
1538745,true,RONCONI Mariano,Male,Head Coach,C,ARG,Argentina,Argentina,Hockey,null,null


In [0]:
# Calculate the total number of rows in the DataFrame
total_rows1 = athletes.count()

# Create a list of columns to drop if they have more than 50% missing values
columns_to_drop1 = []

# Iterate through the columns and check if the number of missing values exceeds 50% of total rows
for column in athletes.columns:
    missing_count1 = athletes.filter(col(column).isNull()).count()  # Count missing (null) values in the column
    if missing_count1 / total_rows1 > 0.5:
        columns_to_drop1.append(column)

# Drop the columns with more than 50% missing values
athletes_cleaned = athletes.drop(*columns_to_drop1)

# Show the cleaned DataFrame
display(athletes_cleaned.limit(10))

code,current,name,name_short,name_tv,gender,function,country_code,country,country_long,nationality,nationality_long,nationality_code,height,weight,disciplines,events,birth_place,birth_country,residence_place,residence_country,occupation,education,family,lang,coach,reason
1532872,true,ALEKSANYAN Artur,ALEKSANYAN A,Artur ALEKSANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0,0,['Wrestling'],"""[""""Men's Greco-Roman 97kg""""]""",GYUMRI,Armenia,GYUMRI,Armenia,Athlete,"Graduated from Shirak State University (Gyumri, ARM)","Father, Gevorg Aleksanyan","Armenian, English, Russian","Gevorg Aleksanyan (ARM), father",He followed his father and his uncle into the sport
1532873,true,AMOYAN Malkhas,AMOYAN M,Malkhas AMOYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0,0,['Wrestling'],"""[""""Men's Greco-Roman 77kg""""]""",YEREVAN,Armenia,YEREVAN,Armenia,null,null,null,Armenian,null,null
1532874,true,GALSTYAN Slavik,GALSTYAN S,Slavik GALSTYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0,0,['Wrestling'],"""[""""Men's Greco-Roman 67kg""""]""",null,null,YEREVAN,Armenia,null,null,null,Armenian,Personal: Martin Alekhanyan (ARM).National: Armen Babalaryan (ARM),null
1532944,true,HARUTYUNYAN Arsen,HARUTYUNYAN A,Arsen HARUTYUNYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0,0,['Wrestling'],"""[""""Men's Freestyle 57kg""""]""",MASIS,Armenia,YEREVAN,Armenia,Athlete,Graduated with a Master's degree from the Armenian State Institute of Physical Culture and Sport (2023),"Wife, Diana (married October 2022). Daughter, Marias (born 2023)",Armenian,National: Habetnak Kurghinyan,"While doing karate he noticed wrestlers training and decided to give it a try. He also tried judo but his father, a former wrestler, did not allow him to do both, so he chose wrestling. (sport.mediamax.am, 10 July 2017)"
1532945,true,TEVANYAN Vazgen,TEVANYAN V,Vazgen TEVANYAN,Male,Athlete,ARM,Armenia,Armenia,Armenia,Armenia,ARM,0,0,['Wrestling'],"""[""""Men's Freestyle 65kg""""]""",POKR VEDI,Armenia,null,Armenia,Athlete,"Studied at the Armenian State Institute of Physical Culture and Sport (Yerevan, ARM)","Wife, Sona (married November 2023)","Armenian, Russian",National: Habetnak Kurghinyan (ARM),"“My family did not like wrestling very much. At first I wanted to do boxing but my older friends advised me to go to wrestling training, and after a week, I started to like the sport.” (myInfo)"
1532951,true,ARENAS Lorena,ARENAS L,Lorena ARENAS,Female,Athlete,COL,Colombia,Colombia,Colombia,Colombia,COL,162,0,['Athletics'],"""[""""Women's 20km Race Walk""""",9/17/93,PEREIRA,Colombia,MELBOURNE,null,Athlete,"Physical Education at Jaime Isaza Cadavid Colombian Polytechnic, Medellin (COL)","Father, Jose Otoniel. Mother, Maria Rudy. Has three siblings",Spanish,Personal: Brent Vallance (AUS)
1533112,true,McKENZIE Ashley,McKENZIE A,Ashley McKENZIE,Male,Athlete,JAM,Jamaica,Jamaica,Jamaica,Jamaica,JAM,0,0,['Judo'],['Men -60 kg'],LONDON,Great Britain,CAMBERLEY,Great Britain,Athlete,null,"One daughter, Lana-Rose, who lives in France",English,Personal and National: Luke Preston (GBR),"""""""I was thrown over [an argument about] a Pokemon card and wanted to learn to throw immediately."""" (Athlete"
1533136,true,BASS BITTAYE Gina Mariam,BASS BITTAYE GM,Gina Mariam BASS BITTAYE,Female,Athlete,GAM,Gambia,Gambia,Gambia,Gambia,GAM,161,0,['Athletics'],"""[""""Women's 100m""""",5/3/95,TUBAKUTA,Gambia,null,null,"Athlete, police officer (sub-inspector)",null,"Husband, Mustapha Bittaye - physical education lecturer and football referee (married October 2023). One sister and two brothers","English, French",Personal: Christophe Belliard (FRA)
1533176,true,CAMARA Ebrahima,CAMARA E,Ebrahima CAMARA,Male,Athlete,GAM,Gambia,Gambia,Gambia,Gambia,GAM,178,0,['Athletics'],"""[""""Men's 100m""""]""",BUNDUNG,Gambia,ANGERS,France,"Athlete, prison officer",null,Married. One daughter. Has three brothers,"Arabic, English, French, Mandinka, Wolof",Personal: Christophe Belliard (FRA),"""""""I 

In [0]:
athletes_cleaned.printSchema()

root
 |-- code: integer (nullable = true)
 |-- current: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- name_short: string (nullable = true)
 |-- name_tv: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- function: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- country_long: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- nationality_long: string (nullable = true)
 |-- nationality_code: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- disciplines: string (nullable = true)
 |-- events: string (nullable = true)
 |-- birth_place: string (nullable = true)
 |-- birth_country: string (nullable = true)
 |-- residence_place: string (nullable = true)
 |-- residence_country: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- education: string (nullable = true)
 |-- family: string (nullable =

In [0]:
#Create a function to drop columns with high null values

def drop_columns_with_high_nulls(df, threshold=0.5):
    total_rows = df.count()  # Total number of rows in the DataFrame
    columns_to_drop = []  # List to hold columns to drop

    for column in df.columns:
        missing_count = df.filter(col(column).isNull()).count()  # Count missing (null) values
        if missing_count / total_rows > threshold:  # Check if nulls exceed the threshold
            columns_to_drop.append(column)

    # Drop the columns with too many missing values
    return df.drop(*columns_to_drop)


In [0]:
drop_columns_with_high_nulls(coaches, threshold=0.5)
drop_columns_with_high_nulls(medals, threshold=0.5)
drop_columns_with_high_nulls(total_medals, threshold=0.5)
total_medals.show(10)

+------------+-------------+--------------------+----------+------------+------------+-----+
|country_code|      country|        country_long|Gold Medal|Silver Medal|Bronze Medal|Total|
+------------+-------------+--------------------+----------+------------+------------+-----+
|         USA|United States|United States of ...|        40|          44|          42|  126|
|         CHN|        China|People's Republic...|        40|          27|          24|   91|
|         JPN|        Japan|               Japan|        20|          12|          13|   45|
|         AUS|    Australia|           Australia|        18|          19|          16|   53|
|         FRA|       France|              France|        16|          26|          22|   64|
|         NED|  Netherlands|         Netherlands|        15|           7|          12|   34|
|         GBR|Great Britain|       Great Britain|        14|          22|          29|   65|
|         KOR|        Korea|   Republic of Korea|        13|          

In [0]:
### drop some columns that i don't need
coaches.drop("events").show(10) 

+-------+-------+-------------------+------+---------------+--------+------------+---------+--------------------+-----------------+----------+
|   code|current|               name|gender|       function|category|country_code|  country|        country_long|      disciplines|birth_date|
+-------+-------+-------------------+------+---------------+--------+------------+---------+--------------------+-----------------+----------+
|1533246|   true|     PEDRERO Ofelia|Female|          Coach|       C|         MEX|   Mexico|              Mexico|Artistic Swimming|1988-03-28|
|1535775|   true|   RADHI SHENAISHIL|  Male|     Head Coach|       C|         IRQ|     Iraq|                Iraq|         Football|1965-07-01|
|1536055|   true|AFLAKIKHAMSEH Majid|  Male|          Coach|       C|         IRI|  IR Iran|Islamic Republic ...|        Taekwondo|1973-08-26|
|1536059|   true|    YOUSEFY Mehrdad|  Male|          Coach|       C|         IRI|  IR Iran|Islamic Republic ...|        Taekwondo|1972-06-12|

In [0]:
athletes_cleaned.repartition(1).write.mode("overwrite").option("header",'true').csv("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/transform-data/athletes_cleaned")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/transform-data/coaches")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/transform-data/medals")
total_medals.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/transform-data/medals-totals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("abfss://paris-olympic-data@parisolympicdataaristide.dfs.core.windows.net/transform-data/teams")